In [27]:
import os
from dotenv import load_dotenv

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
dotenv_path = os.path.join(parent_dir, '.env')
load_dotenv(dotenv_path)

True

In [32]:
from llama_index.core import SimpleDirectoryReader

def load_docs(folder_path: str):
    reader = SimpleDirectoryReader(folder_path, recursive=True)
    docs = reader.load_data(show_progress=True)
    lc_docs = [doc.to_langchain_format() for doc in docs]
    return lc_docs

In [29]:
from langchain_ollama.llms import OllamaLLM
from langchain_ollama import OllamaEmbeddings

CHAT_MODEL = os.getenv("OLLAMA_CHAT_MODEL")
EMBEDDING_MODEL = os.getenv("OLLAMA_EMBEDDING_MODEL")

ollama_llm = OllamaLLM(model=CHAT_MODEL)
ollama_embedding = OllamaEmbeddings(model=EMBEDDING_MODEL)


In [ ]:
from ragas.testset import TestsetGenerator

documents = load_docs("./data")
generator = TestsetGenerator.from_langchain(llm=ollama_llm, embedding_model=ollama_embedding)
dataset = generator.generate_with_langchain_docs(documents, testset_size=100)